# Problem set 2 

## 155 pts

## For problems 1, 2, 4: all functions that you are asked to implement, you have to complete in separate file ```pset2.py```, where we provide signatures of the required functions . Also only this ```py```-file you have to submit in the bot to check correctness of your implementations.

## For problem 3: see instructions in text

## Problem 1 (LU decomposition) 35 pts

### 1. LU for band matrices 

The complexity to find an LU decomposition of a dense $n\times n$ matrix is $\mathcal{O}(n^3)$.
Significant reduction in complexity can be achieved if the matrix has a certain structure, e.g. it is sparse. 
In the following task we consider an important example of $LU$ for a special type of sparse matrices –– band matrices with the bandwidth $m$ equal to 3 or 5 which called tridiagonal and pentadiagonal respectively.

- (5 pts) Write a function ```band_lu(diag_broadcast, n)``` which computes LU decomposition for tridiagonal or pentadiagonal matrix with given diagonal values. 
For example, input parametres ```(diag_broadcast = [1,-2,1], n = 4)``` mean that we need to find LU decomposition for the triangular matrix of the form:

$$A = \begin{pmatrix}
-2 & 1 & 0 & 0\\
1 & -2 & 1 & 0 \\
0 & 1 & -2 & 1 \\
0 & 0 & 1 & -2 \\
\end{pmatrix}.$$

As an output it is considered to make ```L``` and ```U``` - 2D arrays representing diagonals in factors $L$ (```L[0]``` keeps first lower diagonal, ```L[1]``` keeps second lower, ...), and $U$ (```U[:,0]``` keeps main diagonal, ```U[:,1]``` keeps first upper, ...). More details you can find in comments to the corresponding function in ```pset2.py``` 
- (2 pts) Compare execution time of the band LU decomposition using standard function from ```scipy```, i.e. which takes the whole matrix and does not know about its special structure, and band decomposition of yours implementation. Comment on the results.

In [468]:
# Implement function in the ```pset2.py``` file
from pset2 import band_lu
from scipy.sparse import diags # can be used with broadcasting of scalars if desired dimensions are large
import numpy as np

#https://www.webpages.uidaho.edu/~barannyk/Teaching/LU_factorization_tridiagonal.pdf
# Your code is here
def band_lu(diag_broadcast, n):
    
    if len(diag_broadcast) == 3:
        L_2 = [0]
        U_dig = []
        U_up = []
        U_dig.append(diag_broadcast[1])
        for i in range(1, n):
            L_2.append(diag_broadcast[0]/U_dig[i-1])
            U_up.append(diag_broadcast[2])
            U_dig.append(diag_broadcast[1] - L_2[i]*U_up[i-1])
        
        L = [L_2[1:], np.ones([1, n])]

        U_up.append(0)
        U = np.vstack((U_dig, U_up))
        U = U.T
        
    if len(diag_broadcast) == 5:
        
        L_l = [diag_broadcast[1]/diag_broadcast[2]]
        U_k = [diag_broadcast[3]]
        U_diag = [diag_broadcast[2], diag_broadcast[2] - L_l[0]*U_k[0]]
        L_m = [diag_broadcast[0]/U_diag[0], diag_broadcast[0]/U_diag[1]]
        
        U_d = np.array([])
        
        for i in range(1, n):
            U_k.append(diag_broadcast[3] -  L_l[i-1]* diag_broadcast[4] )
            U_diag.append( diag_broadcast[2] - L_m[i-2]*diag_broadcast[4] - L_l[i-1]*U_k[i-1])
            
            U_d= np.append(U_d, diag_broadcast[4])
            
            L_l.append( (diag_broadcast[1] - L_m[i-1]*U_k[i-1])/U_diag[i] )
            
            L_m.append( diag_broadcast[0]/U_diag[i] )
            
        #print(len(L_l), len(U_k), len(U_diag), len(L_m))
        
        L_l.append(0)
        
        L = [L_m, L_l, np.ones([1, n])]

        U_k.append(0)
        
        
        U = np.vstack((U_diag, U_k))
        U_d = np.append(U_d, 0)
        U_d = np.append(U_d, 0)
        
        U_d = U_d.reshape([np.shape(U_d)[0], 1])
        
        
        U = np.vstack((U, U_d.T))
        U = U.T
        

    return L, U   

In [473]:
d = [1, 2, -3, 4, 1]
dd = 6

Ll, Uu = band_lu(d, dd)
print(Uu, Ll)

[[ -3.           4.           1.        ]
 [ -0.33333333   4.66666667   1.        ]
 [  2.66666667  14.           1.        ]
 [ 44.          -2.           1.        ]
 [-84.           3.           1.        ]
 [  2.           4.0327381    0.        ]
 [ -3.27678571   0.           0.        ]] [[-0.3333333333333333, -2.9999999999999987, -2.9999999999999987, 0.3750000000000002, 0.02272727272727274, -0.011904761904761908, 0.5000000000000001], [-0.6666666666666666, -9.999999999999995, 6.000000000000002, 0.9999999999999997, -0.032738095238095254, 0.9659090909090912, 0], array([[1., 1., 1., 1., 1., 1.]])]


In [2]:
diag_broadcast = [1,-2,1]
n = 4
L, U = band_lu(diag_broadcast, n)

In [3]:
L

[[-0.5, -0.6666666666666666, -0.7499999999999999], array([[1., 1., 1., 1.]])]

In [4]:
U

array([[-2.        ,  1.        ],
       [-1.5       ,  1.        ],
       [-1.33333333,  1.        ],
       [-1.25      ,  0.        ]])

In [5]:
import time
import scipy
from scipy.linalg import lu

diag_broadcast = [1,-2,1]
n = 6

matrix = np.array([[-2, 1, 0, 0, 0, 0], [1, -2, 1, 0, 0, 0], [0, 1, -2, 1, 0, 0], [0, 0, 1, -2, 1, 0], 
                   [0, 0, 0, 1, -2, 1], [0, 0, 0, 0, 1, -2]])

start_time = time.clock()
L, U = band_lu(diag_broadcast, n)
print('band_lu =', time.clock() - start_time)

start_time1 = time.clock()
p, l, u = scipy.linalg.lu(matrix)
print('scipy =', time.clock() - start_time1)

band_lu = 0.0001129999999998077
scipy = 0.04722500000000007


How was sad in the task: Significant reduction in complexity can be achieved if the matrix has a certain structure, e.g. it is sparse. On the contrary, the skipy does not know about the special structure of the matrix, which requires more operations. 

Wikipedia: The cost of computation is determined by the number of nonzero entries, rather than by the size of the matrix. (for sparse matrix)

### 2. Completing the proof of existence of LU 

Some details in lecture proofs about $LU$ were omitted. Let us complete them here.
- (5 pts) Prove that if $LU$ decomposition exists, then matrix is strictly regular.
- (5 pts) Prove that if $A$ is a strictly regular matrix, then $A_1 = D - \frac 1a b c^T$ (see lectures for notations) is also strictly regular.

- A matrix $A$ is called strictly regular, if all of its leading principal minors (i.e, submatrices in the first $k$ rows and $k$ columns) are non-singular.

(In the case of the implemented program, assume the converse that there are major minors that are 0. Consider the case, when all diag elements = 0. We get division by 0 already in the first step.)

Consider the case when all elements in the matrix are different.
If $a_{11} = l_{11} u_{11} = 0$, then $l_{11}$ or $u_{11}$ should = 0, that means that the matrices L or U is singular.
We can continue to consider the elements, but as stated in the lecture: "this(that If there is an LU-decomposition, then the matrix is strictly regular) follows from the fact that to get a minor you multiply a corresponding submatrix of $L$ by a corresponding submatrix of $U$, and they are non-singular (since non-singularity of triangular matrices is equivalent to the fact that their diagonal elements are not equal to zero)."

(from lecture) The other way can be proven by induction. Suppose that we know that for all $(n-1) \times (n-1)$ matrices will non-singular minors LU-decomposition exists.
Then, consider the block form $$
    A = \begin{pmatrix}
          a & c^{\top} \\
          b & D
    \end{pmatrix},
$$ where $D$ is $(n-1) \times (n-1)$.
Then we do "block elimination":
$$
     \begin{pmatrix}
     1 & 0 \\
     -z & I
     \end{pmatrix}
     \begin{pmatrix}
     a & c^{\top} \\
     b & D 
     \end{pmatrix}=
     \begin{pmatrix}
     a & c^{\top} \\
     0 & A_1
     \end{pmatrix},
$$
where $z = \frac{b}{a}, \quad A_1 = D - \frac{1}{a} b c^{\top}$.


$A_1$ is also strictly regular, because in the first part we suppose that $(n-1) \times (n-1)$ matrices will non-singular minors LU-decomposition exists.

It has (by induction) the LU-decomposition:
$$
   A_1 = L_1 U_1, 
$$ And that gives the $LU$ decomposition of the original matrix.


- If A is a strictly regular matrix, then it has LU decomposition.

$ A = \begin{pmatrix}
          a & c^{\top} \\
          b & D
    \end{pmatrix} = LU = \begin{pmatrix}
     1 & 0 \\
     z & I
     \end{pmatrix} \begin{pmatrix}
     a & c^{\top} \\
     0 & A_1
     \end{pmatrix}$
     
If matrix $U$ has singular $A_1$, that means that matrix A should be singular too(from the first point), but we take A as a strictly regular matrix. Got a contradiction. $A_1$ is also strictly regular.


### 3. Stability of LU 

Let
$A = \begin{pmatrix}
\varepsilon & 1 & 0\\
1 & 1 & 1 \\
0 & 1 & 1
\end{pmatrix}.$ 
* (5 pts) Find analytically LU decomposition with and without pivoting for the matrix $A$.
* (3 pts) Explain, why can the LU decomposition fail to approximate factors $L$ and $U$ for $|\varepsilon|\ll 1$ in computer arithmetic?

- without pivoting A = LU, L -  unit lower triangular, U -  upper triangular. A should be strictly regular.

if $\varepsilon = 0$ we don't have LU decomposition. 

$\varepsilon \neq 0$

$ 
A = \begin{pmatrix}
\varepsilon & 1 & 0\\
1 & 1 & 1 \\
0 & 1 & 1
\end{pmatrix} =  \begin{pmatrix}
1 & 0 & 0\\
l_1 & 1 & 0 \\
0 & l_2 & 1
\end{pmatrix}  
\begin{pmatrix}
u_1 & 1 & 0\\
0 & u_2 & 1 \\
0 & 0 & u_3
\end{pmatrix}    $ 

$ \varepsilon = u_1 $

$l_1 = \frac{1}{\varepsilon}$

$l_1 + u_2 = 1 \rightarrow u_2 = 1 - \frac{1}{\varepsilon} = \frac{\varepsilon - 1  }{\varepsilon} $

$l_2 u_2 = 1 \rightarrow l_2 = \frac{\varepsilon  }{\varepsilon -1} $

$l_2 + u_3 = 1 \rightarrow u_3 = \frac{-1 }{\varepsilon -1} $

if $\varepsilon = 1$, we get division by zero. So $\varepsilon \neq  1$

if $\varepsilon -> 0$, that $u_2  -> \infty $ and $l_1 - > \infty $





In [491]:
import numpy as np
eps = 1e-18#1.12e-16
a = [[eps, 1, 0],[1.0,  1, 1], [0, 1, 1]]
a = np.array(a)
a0 = a.copy()
n = a.shape[0]
L = np.zeros((n, n))
U = np.zeros((n, n))
for k in range(n): #Eliminate one row   
    L[k, k] = 1
    for i in range(k+1, n):
        L[i, k] = a[i, k] / a[k, k]
        for j in range(k+1, n):
            a[i, j] = a[i, j] - L[i, k] * a[k, j]
    for j in range(k, n):
        U[k, j] = a[k, j]

print('eps = 1e-18')
print('L * U - A:\n', np.dot(L, U) - a0)

eps = 1e-18
L * U - A:
 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.11022302e-16 -1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.11022302e-16  0.00000000e+00]]


In [492]:
import numpy as np
eps = 1e+18
a = [[eps, 1, 0],[1.0,  1, 1], [0, 1, 1]]
a = np.array(a)
a0 = a.copy()
n = a.shape[0]
L = np.zeros((n, n))
U = np.zeros((n, n))
for k in range(n): #Eliminate one row   
    L[k, k] = 1
    for i in range(k+1, n):
        L[i, k] = a[i, k] / a[k, k]
        for j in range(k+1, n):
            a[i, j] = a[i, j] - L[i, k] * a[k, j]
    for j in range(k, n):
        U[k, j] = a[k, j]
print('eps = 1e+18')
print('L * U - A:\n', np.dot(L, U) - a0)

eps = 1e+18
L * U - A:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [493]:
import numpy as np
eps = 1
a = [[eps, 1, 0],[1.0,  1, 1], [0, 1, 1]]
a = np.array(a)
a0 = a.copy()
n = a.shape[0]
L = np.zeros((n, n))
U = np.zeros((n, n))
for k in range(n): #Eliminate one row   
    L[k, k] = 1
    for i in range(k+1, n):
        L[i, k] = a[i, k] / a[k, k]
        for j in range(k+1, n):
            a[i, j] = a[i, j] - L[i, k] * a[k, j]
    for j in range(k, n):
        U[k, j] = a[k, j]
print('eps = 1')
print('L * U - A:\n', np.dot(L, U) - a0)

eps = 1
L * U - A:
 [[ 0.  0. nan]
 [ 0.  0. nan]
 [nan nan nan]]


/Users/anna/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  if sys.path[0] == '':


- with pivoting: A = PLU, P - is a permutation matrix

(not unique, there may be several possible choices for P, L, U)


$ 
P^{-1} A = \begin{pmatrix}
1 & 1 & 1\\
\varepsilon & 1 & 0 \\
0 & 1 & 1
\end{pmatrix} = 
\begin{pmatrix}
1 & 0 & 0\\
l_1 & 1 & 0 \\
0 & l_2 & 1
\end{pmatrix}  
\begin{pmatrix}
u_1 & m_0 & 1\\
0 & u_2 & m_1 \\
0 & 0 & u_3
\end{pmatrix} =  \begin{pmatrix}
u_1 & m_0 + u_1 & 1\\
l_1 u_0  & l_1 m_0 + u_1 & l_1 + m_1 \\
0 & l_2 u_1 & l_2 m_1 + u_2
\end{pmatrix} = \begin{pmatrix}
u_1 & m_0 + u_1 & 1\\
l_1 u_0  & l_1 m_0 + u_1 & l_1 + m_1 \\
0 & l_2 u_1 & l_2 m_1 + u_2
\end{pmatrix} $ 


$ P = \begin{pmatrix}
0 & 1 & 0\\
1 & 0 & 0 \\
0 & 0 & 1
\end{pmatrix} $

$P^{-1} A =  \begin{pmatrix}
1 & 0 & 0\\
\varepsilon & 1 & 0 \\
0 & 1 & 1
\end{pmatrix}  
\begin{pmatrix}
1 & 0 & 1\\
0 & 1 & -\varepsilon \\
0 & 0 & 1+ \varepsilon
\end{pmatrix} $

We can take any epsilon, no special cases


__________
- With LU decomposition, the accuracy of the calculations is critical. The size of the mantissa determines how accurately the numbers can be represented. Consequently, when choosing $|\varepsilon|\ll 1$, we can exceed this accuracy when calculating

### 4. Block LU 

Let $A = \begin{bmatrix} A_{11} & A_{12} \\ A_{21} & A_{22} \end{bmatrix}$ be a block matrix. The goal is to solve the linear system

$$
     \begin{bmatrix} A_{11} & A_{12} \\ A_{21} & A_{22} \end{bmatrix} \begin{bmatrix} u_1 \\ u_2 \end{bmatrix} = \begin{bmatrix} f_1 \\ f_2 \end{bmatrix}.
$$

* (2 pts) Using block elimination find matrix $S$ and right-hand side $\hat{f_2}$ so that $u_2$ can be found from $S u_2 = \hat{f_2}$. Note that the matrix $S$ is called <font color='red'> Schur complement </font> of the block $A_{11}$.
* (4 pts) Using Schur complement properties prove that 

$$\det(X+AB) = \det(X)\det(I+BX^{-1}A), $$


where $X$ - nonsingular square matrix.
* (4 pts) Let matrix $F \in \mathbb{R}^{m \times n}$ and $G \in \mathbb{R}^{n \times m}$. Prove that 

$$\det(I_m - FG) = \det(I_n - GF).$$

- http://www.cs.cornell.edu/~bindel/class/cs6210-f12/notes/lec09.pdf

$\begin{bmatrix} 
A_{11} & A_{12} \\ A_{21} & A_{22} \end{bmatrix}
\begin{bmatrix} u_1 \\ u_2 \end{bmatrix} = \begin{bmatrix} f_1 \\ f_2 \end{bmatrix} $

$ \begin{bmatrix} 
A_{11} & A_{12} \\ A_{21} & A_{22} \end{bmatrix} = \begin{bmatrix} 
L_{11} & 0 \\ L_{21} & L_{22} \end{bmatrix} \begin{bmatrix} 
U_{11} & U_{12} \\ 0 & U_{22} \end{bmatrix} = \begin{bmatrix} 
L_{11} U_{11} & L_{11} U_{12} \\ L_{21} U_{11} & L_{21} U_{12} + L_{22} U_{22} \end{bmatrix} $

$U_{12} = L^{-1}_{11} A_{12}$

$L_{21} = A_{21} U^{-1}_{11} $

$ L_{22} U_{22} = A_{22} - L_{21} U_{12} = A_{22} - A_{21} U^{-1}_{11} L^{-1}_{11} A_{12} = A_{22} - A_{21} A^{-1}_{11} A_{12} = S$

$ \hat{f_2} = f_2 - A_{21} A^{-1}_{11} f_1$

- 
https://terrytao.wordpress.com/2013/01/13/matrix-identities-as-derivatives-of-determinant-identities/#schurd

$M = \begin{bmatrix} 
A & B \\ C & D \end{bmatrix} = \begin{bmatrix} 
A & 0 \\ 0 & 1 \end{bmatrix} \begin{bmatrix} 
1 & A^{-1}B \\ C & D \end{bmatrix} = \begin{bmatrix} 
A & 0 \\ 0 & 1 \end{bmatrix} \begin{bmatrix} 
1 & 0 \\ C & 1 \end{bmatrix} \begin{bmatrix} 
1 & A^{-1}B \\ 0 & D - C A^{-1}B \end{bmatrix} $

$det(M) = det(A)det(D - C A^{-1}B) $

Take $ M = \begin{bmatrix} 
X & A \\ -B & I \end{bmatrix} = \det(X+AB) = \det(X)\det(I+BX^{-1}A) $



- http://www.cari-info.org/cari-2012/session%203/3A2.pdf

Any eigenvalue of $FG$ is either zero or is an eigenvalue of $GF$.

Consider $(FG)u = \lambda u$ that $\lambda \neq 0$ and $u \neq 0$, where $\lambda$ is eigenvalue of $FG$, $u$ is eigenvector. 

$ GFGu = V \lambda u  \rightarrow (GF)(Gu) = \lambda (Gu)$

If $(Gu) = 0 $, then $F(Gu) = \lambda u = 0 \rightarrow \lambda = 0$ 

$\rightarrow (Gu) \neq 0 $


$\rightarrow (\lambda, Gu)$ is eigenpair of $GF$

$\rightarrow$ the nonzero eigenvalues of $FG$ are nonzero eigenvalues of $GF$.

Consider the spectrum of eigenvalues $\Lambda(matrix)$:

$\Lambda(FG) \cup$ {0} $= \Lambda(GF) \cup ${0}, which implies $\Lambda(I_m - FG) \cup ${-1}$ = \Lambda(I_n - GF) \cup ${-1}

Therefore $\det(I_m - FG) = \det(I_n - GF)$




## Problem 2 (QR decomposition) 30 pts

### 1. Standard Gram-Schmidt algorithm
Our goal is to orthogonalize a system of linearly independent vectors $v_1,\dots,v_n$.
The standard algorithm for this task is the Gram-Schmidt process:

$$
\begin{split}
u_1 &= v_1, \\
u_2 &= v_2 - \frac{(v_2, u_1)}{(u_1, u_1)} u_1, \\
\dots \\
u_n &= v_n - \frac{(v_n, u_1)}{(u_1, u_1)} u_1 - \frac{(v_n, u_2)}{(u_2, u_2)} u_2 - \dots - \frac{(v_n, u_{n-1})}{(u_{n-1}, u_{n-1})} u_{n-1}.
\end{split}
$$

Obtained $u_1, \dots, u_n$ are orthogonal vectors in exact arithmetics. Then to make the system orthonormal you should divide each of the vectors by its norm: $u_i := u_i/\|u_i\|$.
The Gram-Schmidt process can be considered as a QR decomposition. Let us show that.

* (2 pts) Write out what is matrices $Q$ and $R$ obtained in the process above. 

* (5 pts) Implement in the ```pset2.py``` the described Gram-Schmidt algorithm as a function ```gram_schmidt_qr(A)``` that takes a rectangular matrix ```A``` and outputs ```Q,R```.

* (3 pts) Create a square [Vandermonde matrix](https://en.wikipedia.org/wiki/Vandermonde_matrix) $V\in\mathbb{R}^{n\times n},\ n = 20$ defined by the vector $x$: ```x = np.linspace(0,1,n)``` (components of $x$ are spaced uniformly between 0 and 1). 
The loss of orthogonality can be described by the following error: $\|Q^{\top}Q-I\|_2$, where $Q^{\top}Q$ is called a Gram matrix. Compute QR decomposition of the created matrix $V$ with function that you have implemented and calculate error $\|Q^{\top}Q-I\|_2$. Comment on the result.

* (5 pts) The observed loss of orthogonality is a problem of this particular algorithm. Luckily, there is [a simple improvement to the algorithm above](https://en.wikipedia.org/wiki/Gram–Schmidt_process#Numerical_stability) that reduces the loss of orthogonality. Implement this modification in the ```pset2.py``` as a function ```modified_gram_schmidt_qr(A)``` such that input and output are similar to ```gram_schmidt_qr(A)```. 
* (3 pts) Compute QR decomposition of the matrix $V$ from the previous task with the function ```modified_gram_schmidt_qr(A)```.
Compute error $\|Q^{\top}Q-I\|_2$. Compare this error to the error obtained with a "pure" Gram-Schmidt and comment on the result.

- Denote $e_i = \frac{u_i}{\|u_i\|} $

$Q$ is an orthogonal matrix and $R$ is an upper triangular matrix.

$ Q = [e_1, ..., e_n] $

$R$ - is a matrix of transition matrix, $R = Q^T A$

In [391]:
# Implement the functions in the ```pset2.py``` file
from pset2 import gram_schmidt_qr
from pset2 import modified_gram_schmidt_qr
import numpy as np

# Your code is here

def gram_schmidt_qr(A):
    
    Q = []
    A = A.transpose()
    m = A.copy()
    u = A

    summ = 0
    for i in range(1, A.shape[0]):
        for j in range(0, i):
            summ = summ + (np.dot(A[i], u[j])/np.dot(u[j],u[j]))*u[j]
        for s in range(0, i):
            u[i] = u[i] - summ
        summ = 0
            
    for k in range(u.shape[0]):
        Q.append(u[k]/(np.linalg.norm(u[k], ord=2)))
    
    R = np.dot(Q, m.transpose())
    
    return np.transpose(Q), R

In [392]:
A = np.array([[12, -51], [6, 167], [-4, 24]])
Q, R = gram_schmidt_qr(A)

In [393]:
Q

array([[ 0.85714286, -0.39428571],
       [ 0.42857143,  0.90285714],
       [-0.28571429,  0.17142857]])

In [394]:
R

array([[ 1.40000000e+01,  2.10000000e+01],
       [-6.66133815e-16,  1.75000000e+02]])

In [395]:
x = np.linspace(0,1, 20)
V = np.vander(x, 20)
    
Q_v, R_v = gram_schmidt_qr(V)
I = np.identity(20)
error1 = np.linalg.norm(np.dot(np.transpose(Q_v), Q_v) - I, ord=2)

In [396]:
error1

17.98773701511802

Wikipedia: The Gram-Schmidt process is inherently numerically unstable. While the application of the projections has an appealing geometric analogy to orthogonalization, the orthogonalization itself is prone to numerical error (due to rounding errors). 

In [397]:
def modified_gram_schmidt_qr(A):
    
    Q = []
    A = A.transpose()
    m = A.copy()
    u = A
    
    for i in range(1, A.shape[0]):
        for j in range(0, i):
            r = np.dot(A[i], u[j])/np.dot(u[j],u[j])
            u[i] = u[i] - r*u[j]
    
    for k in range(u.shape[0]):
        Q.append(u[k]/(np.linalg.norm(u[k], ord=2)))
        
    R = np.dot(Q, m.transpose())
    
    return np.transpose(Q), R

In [398]:
Q_m, R_m = modified_gram_schmidt_qr(V)
error2 = np.linalg.norm(np.dot(np.transpose(Q_m), Q_m) - I, ord=2)

In [399]:
error2

0.9854178703880512

Wikipedia: This approach gives the same result as the original formula in exact arithmetic and introduces smaller errors in finite-precision arithmetic. 

With classical GS, these errors all add up; with modified GS they get eliminated.

### 2. Householder QR (10 pts)

* (7 pts) Implement algorithm for computing QR decomposition based on Householder reflections as a function ```householder_qr(A)``` that takes a rectangular matrix ```A``` and outputs ```Q,R```.


* (2 pts) Apply it to the Vandermonde matrix $V$ created above. Print out the error $\|Q^{\top}Q-I\|_2$, where $Q$ is given by ```householder_qr(A)```. Compare it to the corresponding results of Gram-Schmidt and modified Gram-Schmidt algorithms and comment on it. 



* (3 pts) For values of $n = \{2,25,100,250,500\}$, create a $B\in\mathbb{R}^{n\times n}$ and an upper triangular matrix $R\in\mathbb{R}^{n\times n}$ both filled with standard normal entries. Use ```numpy``` (or ```scipy```) built-in QR decomposition function to obtain a random orthogonal matrix $Q$ from the decomposition of $B$. Then compute $A = QR$ and apply your Gram-Schmidt and Householder algorithms to find the $Q$ and $R$ factors of $A$ – denoted as $\hat{Q}$ and $\hat{R}$. 
Calculate relative errors
$$\frac{\|R-\hat{R}\|_2}{\|R\|_2}, \frac{\|Q-\hat{Q}\|_2}{\|Q\|_2}, \frac{\|A-\hat{Q}\hat{R}\|_2}{\|A\|_2}$$ 
for each value of $n$ and for both algorithms. 
**Note:** scale (multiply corresponding rows/columns by -1) $Q, R,\hat{Q},\hat{R}$ such that diagonal elements of $R$ and $\hat{R}$ be positive.    
    * Comment on the relative errors in $Q$ and $R$ (forward error) compared to ones in $QR$ (backward error).
    * Comment on the backward error obtained for Gram-Schmidt compared to Householder.   

In [518]:
# Implement the function in the pset2.py file
from pset2 import householder_qr

# Your code is here
def householder_qr(A):
    
    A = A.T
    
    Q = np.eye(A.shape[1])
    #P = np.eye(A.shape[0], A.shape[1])
    
    R = A.copy()
    
    AA = A.copy()
    
    for i in range(R.shape[0]-1):
        
        P = np.eye(R.shape[1]) #1])
        I = np.eye(R.shape[1]) #1])
        R = R[i:, i:]
        alfa = np.linalg.norm(R[:1], ord=2)

        e = np.zeros([np.shape(R)[1],])#len(R),])
        e[0] = alfa
        u = R[:1] - np.sign(alfa)*e
        v = u/np.linalg.norm(u, ord=2)
        v = v.reshape((np.shape(v)[1], 1))
        
        P[i:, i:] = I[i:, i:] - 2*np.dot(v, v.T)
        
        R = np.dot(P, AA.T)
        R = R.T
        AA = R.copy()
    
        Q = np.dot(Q, P)
    
    return Q, R.transpose()
        
        

In [519]:
A = np.array([[12, -51], [6, 167], [-4, 24]])
Q, R = householder_qr(A)

In [520]:
Q

array([[ 0.85714286,  0.42857143, -0.28571429],
       [ 0.42857143, -0.28571429,  0.85714286],
       [-0.28571429,  0.85714286,  0.42857143]])

In [521]:
R

array([[ 1.40000000e+01,  2.10000000e+01],
       [ 1.77635684e-15, -4.90000000e+01],
       [-1.33226763e-15,  1.68000000e+02]])

In [400]:
Q_h, R_h = householder_qr(V)
error3 = np.linalg.norm(np.dot(np.transpose(Q_h), Q_h) - I, ord=2)

In [401]:
error3

1.6023591021195354e-15

How we can see, the Householder method gives the smallest error. 

resource https://math.stackexchange.com/questions/770882/why-is-householder-computationally-more-stable-than-modified-gram-schmidt/771271#771271 :


"The main difference is in the accuracy of the orthogonal factor. The Householder orthogonalisation gives $Q$ which is almost orthogonal in the sense that $\|Q^{\top}Q-I\|_2 \leq с\epsilon$. In the modified Gram-Schmidt, this "loss of orthogonality" is proportional to the condition number of $A$ that is $\|Q^{\top}Q-I\|_2 \leq с \epsilon cond_{2}A$

## Problem 3 (Word2Vec as Matrix Factorization) 45 pts

In this assignment you are supposed to apply SVD to training your own [word embedding model](https://en.wikipedia.org/wiki/Word_embedding) which maps English words to vectors of real numbers.

Skip-Gram Negative Sampling (SGNS) word embedding model, commonly known as **word2vec** ([Mikolov et al., 2013](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)), is usually optimized by stochastic gradient descent. However, the optimization of SGNS objective can be viewed as implicit matrix factorization objective as was shown in ([Levy and Goldberg, 2015](http://papers.nips.cc/paper/5477-neural-word-embedding-as-implicit-matrix-factorization.pdf)).

###### 1. Notation
Assume we have a text corpus given as a sequence of words $\{w_1,w_2,\dots,w_n\}$ where $n$ may be larger than $10^{12}$ and $w_i \in \mathcal{V}$ belongs to a vocabulary of words $\mathcal{V}$. A word $c \in \mathcal{V}$ is called *a context* of word $w_i$ if they are found together in the text. More formally, given some measure $L$ of closeness between two words (typical choice is $L=2$), a word $c \in \mathcal{V}$ is called a context if $c \in \{w_{i-L}, \dots, w_{i-1}, w_{i+1}, \dots, w_{i+L} \}$ Let $\mathbf{w},\mathbf{c}\in\mathbb{R}^d$ be the *word embeddings* of word $w$ and context $c$, respectively. Assume they are specified by the mapping  $\Phi:\mathcal{V}\rightarrow\mathbb{R}^d$, so $\mathbf{w}=\Phi(w)$. The ultimate goal of SGNS word embedding model is to fit a good mapping $\Phi$.

Let $\mathcal{D}$ be a multiset of all word-contexts pairs observed in the corpus. In the SGNS model, the probability that word-context pair $(w,c)$ is observed in the corpus is modeled as the following distribution:

$$
P(\#(w,c)\neq 0|w,c) = \sigma(\mathbf{w}^\top \mathbf{c}) = \frac{1}{1 + \exp(-\mathbf{w}^\top \mathbf{c})},
$$

where $\#(w,c)$ is the number of times the pair $(w,c)$ appears in $\mathcal{D}$ and $\mathbf{w}^\top\mathbf{c}$ is the scalar product of vectors $\mathbf{w}$ and $\mathbf{c}$. Two important quantities which we will also use further are the number of times the word $w$ and the context $c$ appear in $\mathcal{D}$, which can be computed as

$$
\#(w) = \sum_{c\in\mathcal{V}} \#(w,c), \quad \#(c) = \sum_{w\in\mathcal{V}} \#(w,c).
$$


###### 2. Optimization objective

Vanilla word embedding models are trained by maximizing log-likelihood of observed word-context pairs, namely

$$
\mathcal{L} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \#(w,c) \log \sigma(\mathbf{w}^\top\mathbf{c}) \rightarrow \max_{\mathbf{w},\mathbf{c} \in \mathbb{R}^d}.
$$

Skip-Gram Negative Sampling approach modifies the objective by additionally minimizing the log-likelihood of random word-context pairs, so called *negative samples*. This idea incorporates some useful linguistic information that some number ($k$, usually $k=5$) of word-context pairs *are not* found together in the corpus which usually results in word embeddings of higher quality. The resulting optimization problem is

$$
\mathcal{L} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \left( \#(w,c) \log \sigma(\mathbf{w}^\top\mathbf{c}) + k \cdot \mathbb{E}_{c'\sim P_\mathcal{D}} \log \sigma (-\mathbf{w}^\top\mathbf{c}) \right) \rightarrow \max_{\mathbf{w},\mathbf{c} \in \mathbb{R}^d},
$$

where $P_\mathcal{D}(c)=\frac{\#(c)}{|\mathcal{D}|}$ is a probability distribution over word contexts from which negative samples are drawn.

[Levy and Goldberg, 2015](http://papers.nips.cc/paper/5477-neural-word-embedding-as-implicit-matrix-factorization.pdf) showed that this objective can be equivalently written as

$$
\mathcal{L} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} f(w,c) = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \left( \#(w,c) \log \sigma(\mathbf{w}^\top\mathbf{c}) + \frac{k\cdot\#(w)\cdot\#(c)}{|\mathcal{D}|} \log \sigma (-\mathbf{w}^\top\mathbf{c}) \right) \rightarrow \max_{\mathbf{w},\mathbf{c} \in \mathbb{R}^d},
$$

A crucial observation is that this loss function depends only on the scalar product $\mathbf{w}^\top\mathbf{c}$ but not on embedding $\mathbf{w}$ and $\mathbf{c}$ separately.

###### 3. Matrix factorization problem statement

Let $|\mathcal{V}|=m$, $W \in \mathbb{R}^{m\times d}$ and $C \in \mathbb{R}^{m\times d}$ be matrices, where each row $\mathbf{w}\in\mathbb{R}^d$ of matrix $W$ is the word embedding of the corresponding word $w$ and each row $\mathbf{c}\in\mathbb{R}^d$ of matrix $C$ is the context embedding of the corresponding context $c$. SGNS embeds both words and their contexts into a low-dimensional space $\mathbb{R}^d$, resulting in the word and context matrices $W$ and $C$. The rows of matrix $W$ are typically used in NLP tasks (such as computing word similarities) while $C$ is ignored. It is nonetheless instructive to consider the product $W^\top C = M$. Viewed this way, SGNS can be described as factorizing an implicit matrix $M$ of dimensions $m \times m$ into two smaller matrices.

Which matrix is being factorized? A matrix entry $M_{wc}$ corresponds to the dot product $\mathbf{w}^\top\mathbf{c}$ . Thus, SGNS is factorizing a matrix in which each row corresponds to a word $w \in \mathcal{V}$ , each column corresponds to a context $c \in \mathcal{V}$, and each cell contains a quantity $f(w,c)$ reflecting the strength of association between that particular word-context pair. Such word-context association matrices are very common in the NLP and word-similarity literature. That said, the objective of SGNS does not explicitly state what this association metric is. What can we say about the association function $f(w,c)$? In other words, which matrix is SGNS factorizing? Below you will find the answers.

### Task 1 (theoretical) 5 pts

Solve SGNS optimization problem with respect to the $\mathbf{w}^\top\mathbf{c}$ and show that the matrix being factorized is

$$
M_{wc} = \mathbf{w}^\top\mathbf{c} = \log \left( \frac{\#(w,c) \cdot |\mathcal{D}|}{k\cdot\#(w)\cdot\#(c)} \right)
$$

**Hint:** Denote $x=\mathbf{w}^\top\mathbf{c}$, rewrite SGNG optimization problem in terms of $x$ and solve it.

**Note:** This matrix is called Shifted Pointwise Mutual Information (SPMI) matrix, as its elements can be written as

$$
\text{SPMI}(w,c) = M_{wc} = \mathbf{w}^\top\mathbf{c} = \text{PMI}(w,c) - \log k
$$

and $\text{PMI}(w,c) = \log \left( \frac{\#(w,c) \cdot |\mathcal{D}|}{\#(w)\cdot\#(c)} \right)$ is the well-known [pointwise mutual information](https://en.wikipedia.org/wiki/Pointwise_mutual_information) of $(w,c)$.

- 
$
\mathcal{L} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} f(w,c) = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \left( \#(w,c) \log \sigma(\mathbf{w}^\top\mathbf{c}) + \frac{k\cdot\#(w)\cdot\#(c)}{|\mathcal{D}|} \log \sigma (-\mathbf{w}^\top\mathbf{c}) \right) = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \left( \#(w,c) \log \sigma(x) + \frac{k\cdot\#(w)\cdot\#(c)}{|\mathcal{D}|} \log \sigma (-x) \right)
$

$\sigma(\mathbf{w}^\top \mathbf{c}) = \frac{1}{1 + \exp(-\mathbf{w}^\top \mathbf{c})},
$


$ \mathcal{L} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \left( \#(w,c) \log (\frac{1}{1 + \exp(-x)}) + \frac{k\cdot\#(w)\cdot\#(c)}{|\mathcal{D}|} \log (\frac{1}{1 + \exp(x)}) \right) = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \left( \#(w,c) (- \log (1 + \exp(-x)) + \frac{k\cdot\#(w)\cdot\#(c)}{|\mathcal{D}|} (-\log (1 + \exp(x)) \right)$

$\frac{\partial \mathcal{L}}{\partial x} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \left( \#(w,c) ( \frac{1}{1 + \exp(-x)} \exp(-x) )- \frac{k\cdot\#(w)\cdot\#(c)}{|\mathcal{D}|} (-\frac{1}{1 + \exp(x)} \exp(x) )\right)  = 0$


$ \#(w,c)  \frac{1}{1 + \exp(-x)}  = \frac{k\cdot\#(w)\cdot\#(c)}{|\mathcal{D}|} \frac{1}{1 + \exp(x)} \exp(2x) $


$  \frac{\#(w,c) |\mathcal{D}|}{ k\cdot\#(w)\cdot\#(c) }  = \frac{1 + \exp(-x)}{1 + \exp(x)} \exp(2x) $

$ \frac{\#(w,c) |\mathcal{D}|}{ k\cdot\#(w)\cdot\#(c) } = \exp(x) $

$ \log (\frac{\#(w,c) |\mathcal{D}|}{ k\cdot\#(w)\cdot\#(c) }) = x = \mathbf{w}^\top\mathbf{c} $

### Task 2 (practical) 40 pts

In [548]:
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

1. Download dataset [enwik8](http://mattmahoney.net/dc/enwik8.zip) of compressed Wikipedia articles and preprocess raw data with Perl script **main_.pl**. This script will clean all unnecessary symbols, make all words to lowercase, and produce only sentences with words.
```
wget http://mattmahoney.net/dc/enwik8.zip
unzip enwik8.zip
mkdir data
perl main_.pl enwik8 > data/enwik8.txt
```

In [547]:
# Load enwik 8

import re
file = open("enwik8.txt", "r")
doclist = [line for line in file]
docstr = ''.join(doclist)
sentences = re.split(r'[.!?]', docstr)
sentences = [sentence.split() for sentence in sentences if len(sentence) > 1]

In [549]:
print (sentences[1249])

['achilles', 'wrath', 'is', 'terrible', 'and', 'he', 'slays', 'many', 'trojan', 'warriors', 'and', 'allies', 'including', 'priam', 's', 'son', 'lycaon', 'whom', 'achilles', 'had', 'previously', 'captured', 'and', 'sold', 'into', 'slavery', 'but', 'who', 'had', 'been', 'returned', 'to', 'troy']


2. Construct the word vocabulary from the obtained sentences which enumerates words which occur more than $r=200$ times in the corpus.

In [561]:
from gensim.models import Word2Vec
import gensim

def create_vocabulary(sentences, r=200):
    vocabulary = {}
    # Your code is here
    model = gensim.models.Word2Vec(sentences, size=150, window=10, min_count=2, workers=10)
    model.train(sentences, total_examples=len(sentences), epochs=10)
    return vocabulary

In [562]:
vocab = create_vocabulary(sentences)

3. Scan the text corpus with sliding window of size $5$ and step $1$ (which corresponds to $L$=2) and construct co-occurrence word-context matrix $D$ with elements $D_{wc}=\#(w,c)$. Please, ignore words which occur less than $r=200$ times, but include them into the sliding window. Please, see the graphical illustration of the procedure described.

![Sliding window](sliding_window.png)

In [ ]:
def create_corpus_matrix(sentences, vocabulary):
    # Your code is here
    return corpus_matrix

In [ ]:
D = create_corpus_matrix(sentences, vocab)

4. To find good word embeddings, [Levy and Goldberg, 2015](http://papers.nips.cc/paper/5477-neural-word-embedding-as-implicit-matrix-factorization.pdf) proposed to find rank-$d$ SVD of Shifted Positive Pointwise Mutual Information (SPPMI) matrix

$$
U \Sigma V^\top \approx \text{SPPMI},
$$

where $\text{SPPMI}(w, c) = \max\left(\text{SPMI}(w, c), 0 \right)$ and $\text{SPMI}(w, c)$ is the element of the matrix $\text{SPPMI}$ at position $(w, c)$.
Then use $W=U\sqrt{\Sigma}$ as word embedding matrix. Your task is to reproduce their results. Write function constructs $\text{SPPMI}$ matrix, computes its SVD and produces word-vectors matrix $W$. Pay attention that $\text{SPPMI}$ matrix is **sparse**!  

In [ ]:
def compute_embeddings(D, k, d=200):
    # Your code is here
    return embedding_matrix

In [ ]:
k = 5 # negative sampling parameter
W = compute_embeddings(D, k)

5. Write class **WordVectors** using provided template.

In [ ]:
class WordVectors:
    
    def __init__(self, vocabulary, embedding_matrix):
        self.vocab = vocabulary
        self.W = embedding_matrix
        self.inv_vocab = {v: k for k, v in self.vocab.items()}
        
    def word_vector(self, word):
        """ 
        Takes word and returns its word vector.
        """
#         Your code is here
        return word_vector
    
    def nearest_words(self, word, top_n=10):
        """ 
        Takes word from the vocabulary and returns its top_n
        nearest neighbors in terms of cosine similarity.
        """
#         Your code is here
        return neighbors

In [ ]:
model = WordVectors(vocab, W)

In [ ]:
model.nearest_words("anarchism")

In [ ]:
model.nearest_words("ussr")

In [ ]:
model.nearest_words("rap")

6. Calculate top 10 nearest neighbours with the corresponding cosine similarities for the words {numerical, linear, algebra} and insert them in the correspoding functions in ```pset2.py```.

# Problem 4 (eigenvalues)  45 pts

### 1. Theoretical tasks

* (5 pts) Prove that normal matrix is Hermitian iff its eigenvalues are real. Prove that normal matrix is unitary iff its eigenvalues satisfy $|\lambda| = 1$. 

* (5 pts) The following problem illustrates instability of the Jordan form. Find theoretically the eigenvalues of the perturbed Jordan block:

$$
    J(\varepsilon) = 
    \begin{bmatrix} 
     \lambda & 1 & & & 0 \\ 
     & \lambda & 1 & & \\ 
     &  & \ddots & \ddots & \\ 
     & & & \lambda & 1 \\ 
     \varepsilon & & & & \lambda  \\ 
    \end{bmatrix}_{n\times n}
$$

Comment how eigenvalues of $J(0)$ are perturbed for large $n$.

- If eigenvalues are real, A is normal matrix. 

*Proposition.* A matrix A is normal if and only if there exists a diagonal matrix $\Lambda$ and a unitary matrix U such that $A = U \Lambda U^*$.

$A^* = (U \Lambda U^*)^* = U (U \Lambda)^* = U \Lambda^* U^* = U \Lambda U^* = A$, as $\Lambda = \Lambda^*$ - the diagonal matrix with real eigenvalues of A.

If A is a normal Hermitian matrix, consequently eigenvalues of matrix A are real, because it's the property of Hermitian matrices. 



 - https://www.utwente.nl/en/eemcs/sacs/teaching/Thesis/beek.pdf

$ det(J(\epsilon)) = \lambda det \begin{bmatrix}  \lambda & 1 & & & 0 \\ &  & \ddots & \ddots & \\ &  & \ddots & \ddots & \\ 0 & & & & \lambda  \\ 
    \end{bmatrix}_{(n-1)\times (n-1)} - det \begin{bmatrix}  0 & 1 & & & 0 \\ 0 & \lambda & 1 & & 0 \\ &  & \ddots & \ddots & \\ \epsilon & & & &   \\ 
    \end{bmatrix}_{(n-1)\times (n-1)} = \lambda^n - (-1)^m \epsilon det \begin{bmatrix}  0 & 1 & & & 0 \\ 0 & \lambda & 1 & & 0 \\ &  & \ddots & \ddots & \\ & & & &   \\ 
    \end{bmatrix} =$
    
    
$= \lambda^n + (-1)^{m+1} \epsilon det \begin{bmatrix}  0 & 1 & & & 0 \\ 0 & \lambda & 1 & & 0 \\ &  & \ddots & \ddots & \\ & & & &   \\ \end{bmatrix} = \lambda^n + (-1)^{n+1}\epsilon = 0 $


$ \lambda^n = (-1)^n \epsilon $

$ \lambda = - \epsilon^{1/n} $

$ \lambda = - \sqrt[n]{\epsilon} $

$\epsilon > 0$, if $ n -> \infty $ : $ \lambda -> -1 $

____
PS. if I misunderstood the task and the eigenvalues had a minus ($ -\lambda $), then the output is the same, only $ \lambda = \sqrt[n]{\epsilon}$ and $if n -> \infty $ : $ \lambda -> 1 $
_____

### 2. PageRank


#### Damping factor importance

* (5 pts) Write the function ```pagerank_matrix(G)``` that takes an adjacency matrix $G$ (in both sparse and dense formats) as an input and outputs the corresponding PageRank matrix $A$.

* (3 pts) Find PageRank matrix $A$ that corresponds to the following graph: <img src="graph.png" width='250'>
What is its largest eigenvalue? What multiplicity does it have?


* (5 pts) Implement the power method for a given matrix $A$, an initial guess $x_0$ and a number of iterations ```num_iter```. It should be organized as a function ```power_method(A, x0, num_iter)``` that outputs approximation to eigenvector $x$, eigenvalue $\lambda$ and history of residuals $\{\|Ax_k - \lambda_k x_k\|_2\}$. Make sure that the method conveges to the correct solution on a matrix $\begin{bmatrix} 2 & -1 \\ -1 & 2 \end{bmatrix}$ which is known to have the largest eigenvalue equal to $3$.


* (2 pts) Run the power method for the graph presented above and plot residuals $\|Ax_k - \lambda_k x_k\|_2$ as a function of $k$ for ```num_iter=100``` and random initial guess ```x0```.  Explain the absence of convergence. 


* (2 pts) Consider the same graph, but with a directed edge that goes from the node 3 to the node 4 being removed. Plot residuals as in the previous task and discuss the convergence. Now, run the power method with ```num_iter=100``` for 10 different initial guesses and print/plot the resulting approximated eigenvectors. Why do they depend on the initial guess?


In order to avoid this problem Larry Page and Sergey Brin [proposed](http://ilpubs.stanford.edu:8090/422/1/1999-66.pdf) to use the following regularization technique:

$$
A_d = dA + \frac{1-d}{N} \begin{pmatrix} 1 & \dots & 1 \\ \vdots & & \vdots \\ 1 & \dots & 1 \end{pmatrix},
$$

where $d$ is a small parameter in $[0,1]$ (typically $d=0.85$), which is called **damping factor**, $A$ is of size $N\times N$. Now $A_d$ is the matrix with multiplicity of the largest eigenvalue equal to 1. 
Recall that computing the eigenvector of the PageRank matrix, which corresponds to the largest eigenvalue, has the following interpretation. Consider a person who stays in a random node of a graph (i.e. opens a random web page); at each step s/he follows one of the outcoming edges uniformly at random (i.e. opens one of the links). So the person randomly walks through the graph and the eigenvector we are looking for is exactly his/her stationary distribution â€” for each node it tells you the probability of visiting this particular node. Therefore, if the person has started from a part of the graph which is not connected with the other part, he will never get there.  In the regularized model, the person at each step follows one of the outcoming links with probability $d$ OR teleports to a random node from the whole graph with probability $(1-d)$.

* (2 pts) Now, run the power method with $A_d$ and plot residuals $\|A_d x_k - \lambda_k x_k\|_2$ as a function of $k$ for $d=0.97$, ```num_iter=100``` and a random initial guess ```x0```.

* (5 pts) Find the second largest in the absolute value eigenvalue of the obtained matrix $A_d$. How and why is it connected to the damping factor $d$? What is the convergence rate of the PageRank algorithm when using damping factor?

Usually, graphs that arise in various areas are sparse (social, web, road networks, etc.) and, thus, computation of a matrix-vector product for corresponding PageRank matrix $A$ is much cheaper than $\mathcal{O}(N^2)$. However, if $A_d$ is calculated directly, it becomes dense and, therefore, $\mathcal{O}(N^2)$ cost grows prohibitively large for  big $N$.


* (2 pts) Implement fast matrix-vector product for $A_d$ as a function ```pagerank_matvec(A, d, x)```, which takes a PageRank matrix $A$ (in sparse format, e.g., ```csr_matrix```), damping factor $d$ and a vector $x$ as an input and returns $A_dx$ as an output. 

* (1 pts) Generate a random adjacency matrix of size $10000 \times 10000$ with only 100 non-zero elements and compare ```pagerank_matvec``` performance with direct evaluation of $A_dx$.

In [544]:
# implement the functions in the pset2.py file
from pset2 import pagerank_matrix, power_method, pagerank_matvec

# http://tuvalu.santafe.edu/~aaronc/courses/5454/csci5454_spring2013_CSL11.pdf
# Code from https://en.wikipedia.org/wiki/PageRank
'''
def pagerank_matrix(G):
    eps=1.0e-8
    d=0.85

    N = np.shape(G)[1]
    v = np.random.rand(N, 1)
    v = v / np.linalg.norm(v, 1)
    last_v = np.ones((N, 1), dtype=np.float32) * 100
    M_hat = (d * G) + (((1 - d) / N) * np.ones((N, N), dtype=np.float32))
    while np.linalg.norm(v - last_v, 2) > eps:
        last_v = v
        v = np.matmul(M_hat, v)
       
    A = []
    PR = []
    for i in range(0, N-1):
        A.append(1/N)
        
    for i in range(0, N-1):    
        PR.append(1-d)
        for j in range(len(G)):
            
        
    return v

'''
    
    

In [545]:
G = np.array([[0, 0, 0, 0, 1],
              [0.5, 0, 0, 0, 0],
              [0.5, 0, 0, 0, 0],
              [0, 1, 0.5, 0, 0],
              [0, 0, 0.5, 1, 0]])
A = pagerank_matrix(G)

In [546]:
A

array([[0.25419172],
       [0.13803148],
       [0.13803148],
       [0.20599012],
       [0.26375497]])

#### DBLP: computer science bibliography

Download the dataset from [here](https://goo.gl/oZVxEa), unzip it and put `dblp_authors.npz`  and `dblp_graph.npz` in the same folder with this notebook. Each value (author name) from `dblp_authors.npz` corresponds to the row/column of the matrix from `dblp_graph.npz`. Value at row `i` and column `j` of the matrix from `dblp_graph.npz` corresponds to the number of times author `i` cited papers of the author `j`. Let us now find the most significant scientists according to PageRank model over DBLP data.

* (4 pts) Load the weighted adjacency matrix and the authors list into Python using ```load_dblp(...)``` function. Print its density (fraction of nonzero elements). Find top-10 most cited authors from the weighted adjacency matrix. Now, make all the weights of the adjacency matrix equal to 1 for simplicity (consider only existence of connection between authors, not its weight). Obtain the PageRank matrix $A$ from the adjacency matrix and verify that it is stochastic.
 
 
* (1 pts) In order to provide ```pagerank_matvec``` to your ```power_method``` (without rewriting it) for fast calculation of $A_dx$, you can create a ```LinearOperator```: 
```python
L = scipy.sparse.linalg.LinearOperator(A.shape, matvec=lambda x, A=A, d=d: pagerank_matvec(A, d, x))
```
Calling ```L@x``` or ```L.dot(x)``` will result in calculation of ```pagerank_matvec(A, d, x)``` and, thus, you can plug $L$ instead of the matrix $A$ in the ```power_method``` directly. **Note:** though in the previous subtask graph was very small (so you could disparage fast matvec implementation), here it is very large (but sparse), so that direct evaluation of $A_dx$ will require $\sim 10^{12}$ matrix elements to store - good luck with that (^_<).


* (2 pts) Run the power method starting from the vector of all ones and plot residuals $\|A_dx_k - \lambda_k x_k\|_2$  as a function of $k$ for $d=0.85$.


* (1 pts) Print names of the top-10 authors according to PageRank over DBLP when $d=0.85$. Comment on your findings.

In [ ]:
from scipy.sparse import load_npz
import numpy as np
def load_dblp(path_auth, path_graph):
    G = load_npz(path_graph).astype(float)
    with np.load(path_auth) as data: authors = data['authors']
    return G, authors
G, authors = load_dblp('dblp_authors.npz', 'dblp_graph.npz')

In [ ]:
# Your code is here